<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Lab: Cleaning Rock Song Data

_Authors: Dave Yerrington (SF)_

---


In [6]:
!python3.8 -m pip install seaborn

     |████████████████████████████████| 225kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 28.8MB 1.8MB/s  eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
import pandas as pd
import numpy as np 
import seaborn as sns

%matplotlib inline

### 1. Load `rock.csv` and do an initial examination of its data columns.

In [8]:
rockfile = "./rock.csv"

In [16]:
# Load the data.
old_rocks = pd.read_csv(rockfile)

In [17]:
# Look at the information regarding its columns.
old_rocks.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


### 2.  Clean up the column names.

Let's clean up the column names. There are two ways we can accomplish this:

#### 2.A Change the column names when you import the data using `pd.read_csv()`.

Notice that, when passing `names=[..A LIST OF STRING..]` with a number of columns that matches the number of strings in the passed list, you replace the column names.

NOTE: When you create custom column names, the first row of the `.csv` already represents a header. It is important to tell `pandas` to skip that row. The `skiprows=1` keyword argument to `read_csv()` will tell `pandas` to skip the first row.

In [18]:
# Change the column names when loading the '.csv':

new_name = ["song_clean", "artist_clean", "release_year", "combined", "first"," year", "play_count", "fg"]

rocks = pd.read_csv(rockfile, header=0, names=new_name)

rocks.head()

,song_clean,artist_clean,release_year,combined,first,year,play_count,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


#### 2.B Change column names using the `.rename()` function.

The `.rename()` function takes an argument, `columns=name_dict`, in which `name_dict` is a dictionary containing the original column names as keys and the new column names as values.

In [22]:
# Change the column names using the `.rename()` function.

rocks.rename(columns={"fg": "FG"})


,song_clean,artist_clean,release_year,combined,first,year,play_count,FG
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,NaN,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


#### 2.C Reassigning the `.columns` attribute of a DataFrame.

You can also just reassign the `.columns` attribute to a list of strings containing the new column names. 

The only caveat with reassigning `.columns` is that you have to reassign all of the column names at once. You can't partially replace a value by working on `.columns` directly. You have to reassign `.columns` with a list of equal length. 

In [28]:
# Replace the column names by reassigning the `.columns` attribute.

reassign_rock = pd.read_csv(rockfile)

reassign_rock.columns = ["Song", "Artist", "Release", "Combined", "First",
                      "Year", "Play", "FG"]
reassign_rock.head()

,Song,Artist,Release,Combined,First,Year,Play,FG
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


### 3. Subsetting data where null values exist.

We have mixed `str` and `NaN` values in the `release` column. `NaN` stands for "not a number" and is the way `pandas` handles "nulls" or nonexistent data. We can use the `.isnull()` method of a Series to find null values.

Print the header of the data subset to where the `release` column is null values.

In [35]:
# Show records where df['release'] is null

rocks[rocks["release_year"].isnull()]

,song_clean,artist_clean,release_year,combined,first,year,play_count,fg
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
10,"Baby, Please Don't Go",AC/DC,NaN,"Baby, Please Don't Go by AC/DC",1,0,1,0
13,CAN'T STOP ROCK'N'ROLL,AC/DC,NaN,CAN'T STOP ROCK'N'ROLL by AC/DC,1,0,5,0
16,Girls Got Rhythm,AC/DC,NaN,Girls Got Rhythm by AC/DC,1,0,24,0
24,Let's Get It Up,AC/DC,NaN,Let's Get It Up by AC/DC,1,0,4,0
...,...,...,...,...,...,...,...,...
2216,"I'm Bad, I'm Nationwide",ZZ Top,NaN,"I'm Bad, I'm Nationwide by ZZ Top",1,0,10,0
2218,Just Got Paid,ZZ Top,NaN,Just Got Paid by ZZ Top,1,0,2,0
2221,My Head's In Mississippi,ZZ Top,NaN,My Head's In Mississippi by ZZ Top,1,0,1,0
2222,Party On The Patio,ZZ Top,NaN,Party On The Patio by ZZ Top,1,0,14,0


### 4. Update slices of your DataFrame based on mask selection/slices.

In many scenarios, we want to upate values in our DataFrame according to criteria. Let's say we wanted to set all of the null values in `release` to 0.

With newer versions of `pandas`, in order to manipulate data in the original DataFrame, we have to use `.loc` while performing reassignment using a mask and an index.

For example, the following won't always work:
```python
df[row_mask]['column_name'] = new_value
```

The best way to accomplish the same task is:
```python
df.loc[row_mask, 'column_name'] = new_value
```

For multiple column assignment, you would use:
```python
df.loc[row_mask, ['col_1', 'col_2', 'col_3']] = new_value
```

#### 4.A Let's try it out. Make all of the null values in `release` 0.

In [41]:
# Replace release nulls with 0
release_null = rocks.release_year.isnull()
rocks[release_null].release_year = 0


#### 4.B Verify that `release` contains no null values.

In [42]:
# A:

rocks.release_year.isnull().sum()

0

### 5. Ensure that the data types of the columns make sense. 

Verifying column data types is a critical part of data munging. If columns have the wrong data type, then there is usually corrupted or incorrect data in some of the observations.

#### 5.A Look at the data types for the columns. Are any incorrect given what the data represents?

In [43]:
# A:
rocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2230 entries, 0 to 2229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   song_clean    2230 non-null   object
 1   artist_clean  2230 non-null   object
 2   release_year  2230 non-null   object
 3   combined      2230 non-null   object
 4   first         2230 non-null   int64 
 5    year         2230 non-null   int64 
 6   play_count    2230 non-null   int64 
 7   fg            2230 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 139.5+ KB


### 6. Investigate and clean up the `release` column.

The `release` column is a string data type when it should be an integer.

#### 6.A Figure out what value(s) are causing the `release` column to be encoded as a string instead of an integer.

In [46]:
# A:

rocks.release_year.value_counts()
rocks.release_year.unique()

array(['1982', 0, '1981', '1980', '1975', '2000', '2002', '1992', '1985',
       '1993', '1976', '1995', '1979', '1984', '1977', '1990', '1986',
       '1974', '2014', '1987', '1973', '2001', '1989', '1997', '1971',
       '1972', '1994', '1970', '1966', '1965', '1983', '1955', '1978',
       '1969', '1999', '1968', '1988', '1962', '2007', '1967', '1958',
       '1071', '1996', '1991', '2005', '2011', '2004', '2012', '2003',
       '1998', '2008', '1964', '2013', '2006', 'SONGFACTS.COM', '1963',
       '1961'], dtype=object)

#### 6.B Look at the rows in which there is incorrect data in the `release` column.

In [47]:
# A:

abnormal = (rocks.release_year == "SONGFACTS.COM") | (rocks.release_year == "1071")
rocks[abnormal]

,song_clean,artist_clean,release_year,combined,first,year,play_count,fg
547,Levon,Elton John,1071,Levon by Elton John,1,1,8,8
1504,Bullfrog Blues,Rory Gallagher,SONGFACTS.COM,Bullfrog Blues by Rory Gallagher,1,1,1,1


#### 6.C. Clean up the data. Normally we may replace the offending data with null np.nan values, however we previously converted all of the nan values in the release column to zeros so we might as well continue with the same practice. Replacing with 0 (or nan) will allow us to convert the column to numeric.

In [50]:
# A:

rocks.release_year[abnormal] = 0

rocks.release_year.unique()

<ipython-input-50-02ccc87e4a74>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rocks.release_year[abnormal] = 0


array(['1982', 0, '1981', '1980', '1975', '2000', '2002', '1992', '1985',
       '1993', '1976', '1995', '1979', '1984', '1977', '1990', '1986',
       '1974', '2014', '1987', '1973', '2001', '1989', '1997', '1971',
       '1972', '1994', '1970', '1966', '1965', '1983', '1955', '1978',
       '1969', '1999', '1968', '1988', '1962', '2007', '1967', '1958',
       '1996', '1991', '2005', '2011', '2004', '2012', '2003', '1998',
       '2008', '1964', '2013', '2006', '1963', '1961'], dtype=object)

In [51]:
rocks.release_year = rocks.release_year.apply(int)
rocks.release_year.unique()

array([1982,    0, 1981, 1980, 1975, 2000, 2002, 1992, 1985, 1993, 1976,
       1995, 1979, 1984, 1977, 1990, 1986, 1974, 2014, 1987, 1973, 2001,
       1989, 1997, 1971, 1972, 1994, 1970, 1966, 1965, 1983, 1955, 1978,
       1969, 1999, 1968, 1988, 1962, 2007, 1967, 1958, 1996, 1991, 2005,
       2011, 2004, 2012, 2003, 1998, 2008, 1964, 2013, 2006, 1963, 1961])

### 7. Get summary statistics for the `release` column using the `.describe()` function.

Now that the `release` column is finally a numeric data type, we can apply the `.describe()` function.  

#### 7.A Print out the summary stats for the `release` column. What is the earliest and latest release date?

In [53]:
# A:

rocks[rocks.release_year > 0].release_year.describe()

count    1651.000000
mean     1978.569352
std         9.309780
min      1955.000000
25%      1971.000000
50%      1977.000000
75%      1984.000000
max      2014.000000
Name: release_year, dtype: float64

#### 7.B Based on the summary statistics, is there anything else wrong with the `release` column? 

In [16]:
# A:

_Looking at the DataFrame that contains the year 1071, we can see that the year was probably corrupted and should be replaced with something else if possible._

### 8. Make changes and investigate using custom functions with `.apply()`.

Let's say we want to traverse every single row in our data set and apply a function to that row.

#### 8.A Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970.


In [56]:
# A:

def print_song_info(row):
    print ("Song is {} by {}. Before 1970: {}". format(row.song_clean, row.artist_clean, row.before_70))

#### 8.B Using the `.apply()` function, apply the function you wrote to the first four rows of the DataFrame.

You will need to tell the `apply` function to operate row by row. Setting the keyword argument as `axis=1` indicates that the function should be applied to each row individually.

In [60]:
# A:

rocks.iloc[[0,1,2,3], :].apply(print_song_info, axis=1);

Song is Caught Up in You by .38 Special. Before 1970: False
Song is Fantasy Girl by .38 Special. Before 1970: True
Song is Hold On Loosely by .38 Special. Before 1970: False
Song is Rockin' Into the Night by .38 Special. Before 1970: False


You'll notice that there will be a final output Series of `None` values. The `.apply()` function, if a return value is not specified, will return a Series of `None` values (similar to how the default return for Python functions is `None` when a return statement is not specified).

### 9. Write a function that converts cells in a DataFrame to float and otherwise replaces them with `np.nan`.

If applied to our data, it would keep only the numeric information and otherwise input null values.

Recall that the try-except syntax in Python is a great way to try something and take another action if the initial step fails:

```python
try:
    Perform some action.
except:
   Perform some other action if the first failed with an error.
```

#### 9.A Write the function that takes a column and converts all of its values to float if possible and `np.nan` otherwise. The return value should be the converted Series.

In [19]:
# A:

#### 9.B Try your function out on the rock song data and ensure the output is what you expected.


In [20]:
# A:

#### 9.C Describe the new float-only DataFrame.

In [21]:
# A: